In [1]:
import numpy as np
from pr3_utils import *
from stereo import *

### load data

In [2]:
dataset = "03"
time_stamp,features,linear_velocity,angular_velocity,k,b,imu_T_cam = \
    load_data(f"../data/{dataset}.npz")
T = np.load(f"../data/predict_T_mean_{dataset}.npy")

### pre-compute all feature init values

In [ ]:
seen_mask = np.zeros(features.shape[1], dtype=bool)
M_init = np.zeros([features.shape[1], 3])

for t in range(time_stamp.shape[0]):
    present_mask = get_seeing_mask(features[t])
    first_time_mask = np.logical_and((np.logical_not(seen_mask)), present_mask)

    PC_camera = features_to_PC(features[t, first_time_mask, :], k, b)
    PC_camera_hom = np.hstack([PC_camera, np.ones([PC_camera.shape[0],1])])
    PC_world_hom  = (T[t] @ imu_T_cam @ PC_camera_hom.T).T
    PC_world = PC_world_hom[:,:3]/PC_world_hom[:,[3]]

    M_init[first_time_mask] = PC_world
    seen_mask[first_time_mask] = True
    
    if np.all(seen_mask):
        print(f"seen all features at {t} stamp")
        break

In [ ]:
fig, _ = visualize_trajectory(T, features=M_init, path_name="EKF_localization", show_ori=False)
fig.savefig(f'../img/EKF_mapping_init_{dataset}.png', dpi=300)
plt.show()

In [ ]:
np.save(f"../data/M_init_{dataset}",M_init)
# features_init = np.load(f"../data/feature_init__{dataset}.npy")
# features_init.shape

### examine one time stamp

In [ ]:
M_init[0]

In [ ]:
p = features_to_PC(features[10,0,:], k, b)
features[10,0,:], p

In [ ]:
p = np.array([*p, 1])
fsu = k[0,0]
fsv = k[1,1]
cu  = k[0,2]
cv  = k[1,2]
Ks = np.array([
    [fsu,0,cu,0],
    [0,fsv,cv,0],
    [fsu,0,cu,-fsu*b],
    [0,fsv,cv,0],
])
Ks @ projection(p)

In [ ]:
present_mask = get_seeing_mask(features[1], d_min=3)
PC_camera = features_to_PC(features[0, present_mask, :], k, b)

fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot
scatter = ax.scatter(PC_camera[:,0], PC_camera[:,1], PC_camera[:,2])

# Labels and titles
ax.set_title('3D Interactive Point Cloud')
ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')
ax.set_zlabel('Z Axis')

# Show plot
plt.show()

# import open3d as o3d
# point_cloud = o3d.geometry.PointCloud()
# point_cloud.points = o3d.utility.Vector3dVector(PC_camera)

# # Optionally, you can color the points. Here, all points are colored red.
# colors = [[1, 0, 0] for i in range(len(PC_camera))]  # Red color in RGB
# point_cloud.colors = o3d.utility.Vector3dVector(colors)
# coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])

# # Visualizing the point cloud
# o3d.visualization.draw_geometries([point_cloud, coordinate_frame])

### estimate W

In [7]:
features_to_PC(features[:,0,:], k, b).shape

/Users/weixiaozhan/GradSchool/24WT/ECE276A_SensingEstimationRobotics/PR3/code/stereo.py:17: RuntimeWarning: divide by zero encountered in divide
  z = fsu * b / d


(1010, 3)